In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [5]:
df=pd.read_csv(r'/content/hair_loss.csv')

X=df.drop(['hair_fall','body_water_content','stress_level','liver_data'],axis=1)
y=df['hair_fall']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
from sklearn.model_selection import RandomizedSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'colsample_bytree': [0.3, 0.7]
}
random_search = RandomizedSearchCV(XGBClassifier(), param_distributions=param_grid, cv=5, n_iter=10, random_state=42)
random_search.fit(X_train, y_train)
print(random_search.best_params_)

{'n_estimators': 300, 'max_depth': 5, 'learning_rate': 0.01, 'colsample_bytree': 0.7}


In [11]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

In [12]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC

voting_clf = VotingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
        ('svm', SVC(probability=True, random_state=42)),
        ('gb', GradientBoostingClassifier(random_state=42))
    ],
    voting='soft'
)

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.18      0.17      0.17      5092
           1       0.17      0.18      0.18      5025
           2       0.17      0.19      0.18      4993
           3       0.19      0.17      0.18      5162
           4       0.18      0.18      0.18      5077
           5       0.16      0.17      0.17      4987

    accuracy                           0.18     30336
   macro avg       0.18      0.18      0.18     30336
weighted avg       0.18      0.18      0.18     30336

